In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('news-items.csv')

In [12]:
# make "group" column int type where its not NaN
df['group'] = df['group'].fillna(0).astype(int)

In [13]:
df

,group,id,label,title,subtitle,content,source,publishing_date,category,highlighted_content,...,chatgpt_explanation,is_false_positive,is_true_negative,is_qualification,control_question,answer_correct,answer_wrong_1,answer_wrong_2,answer_wrong_3,answer_wrong_4
0,1,1 health: 1,TRUE,‘Miracle’ baby opens debate over possible use ...,Technology allows sperm to be frozen longer th...,A change of law has paved the way for more bab...,Trustworthy Source,10/28/2022,Health,A change of law has paved the way for more bab...,...,This article discusses a real and recent event...,False,False,False,What was the article about?,A change of law about frozen sperm up to 50 ye...,A change of law about about frozen babies,A change of law about frozen animals,Frozen sperm should be made freely available f...,Frozen sperm has become a new ice cream
1,1,1 health: 2,FAKE,All Homeopathic Products Now Illegal?,NaN,About 5 million U.S. adults and 1 million U.S....,Untrustworthy Source,1/30/2018,Health,About 5 million U.S. adults and 1 million U.S....,...,While the article presents some factual inform...,False,False,False,What was the article about?,The legal status of homeopathic products,The legal status of cancer treatments,The legal status of forbidden drugs,Drugs should become freely available,Children should receive cancer treatments
2,1,1 health: 3,FAKE,Young adult cannabis consumers nearly twice as...,NaN,"(CNN) — Whether you smoke it, vape it or eat i...",Untrustworthy Source,9/7/2021,Health,"Whether you smoke it, vape it or eat it as an ...",...,This article cites a published research study ...,False,False,False,What was the article about?,Young adult cannabis consumers suffer from a h...,Young children consuming cannabis suffer from ...,Creazy cannabis consumers robbing shops more l...,Cannabis is healthy for your merriage,Cannabis can be consumed anytime in Alaska
3,1,2 immi: 4,TRUE,Borders watchdog left ‘speechless’ by failings...,Chief inspector told MPs that Manston processi...,The borders watchdog said he was left speechle...,Trustworthy Source,10/26/2022,Immigration,The borders watchdog said he was <mark>left sp...,...,This article reports on real events and statem...,False,False,False,What was the article about?,Borders watchdog left ‘speechless’ by failings...,Border watchdog eats hot dog in Kent,Border watchdog helps refugess to cook a soup,Refugees are staying in Kent for vacation,Kent welcomes all refugees to stay there as lo...
4,1,2 immi: 5,FAKE,"10,800 U.S. Children Were Raped by Illegal Imm...",The number of U.S. children raped by illegal i...,What is it going to take to get a wall built o...,Untrustworthy Source,12/30/2018,Immigration,"Tucker Carlson had the former mayor of Shiloh,...",...,There is no credible source to support the cla...,False,False,False,What was the article about?,U.S. children were raped by illegal immigrants,French children were raped by robbers in the U.S.,Children were raped by extratererestrial being...,Illegal immigrants cross border to Mexico,Canadad suffers from illegal immigrants the most
5,1,2 immi: 6,FAKE,REPORT: One-Third Of Migrant Caravan Sick: HIV...,NaN,Thousands of migrants marched from Central Ame...,Untrustworthy Source,11/29/2018,Immigration,Thousands of migrants marched from Central Ame...,...,While it is true that many migrants in the car...,False,False,False,What was the article about?,"One-third of migrant caravan sick: HIV, Tuberc...","One-third of alians are sick, detected juts re...",One-third of illegal immigrants are sick in th...,Illegal immigrants spread dangerous deseas,If you are immigrant you are more likely to ge...
6,2,3 env: 1,TRUE,"Facing unbearable heat, Qatar has begun to air...",2°C: BEYOND THE LIMIT,"Already one of the hottest places on Earth, Qa...",Trustworthy Source,10/16/2019,Environment,"Already one of the hottest places on Earth, Qa...",...,This article reports on the extreme heat in Qa...,False,False,False,What was the article about?,"Facing unbearable heat, Qata

In [14]:
# replace id values with index values
df['id'] = df.index + 1

In [15]:
news_items = []

for index, row in df.iterrows():
    news_items.append({
        'id': row['id'],
        'group': row['group'],
        'isFalsePositive': row['is_false_positive'],
        'isTrueNegative': row['is_true_negative'],
        'isQualification': row['is_qualification'],
        'title': row['title'],
        # make null if NaN
        'subtitle': row['subtitle'] if not pd.isnull(row['subtitle']) else None,
        'content': row['content'],
        'source': row['source'],
        'category': row['category'],
        'publishingDate': row['publishing_date'],
        'label': row['label'].lower(),
        'xaiFeatures': {
            'truthfulness': int(row['truthfulness'].replace('%', '')),
            'readability': row['readability'],
            'highlightedContent': row['highlighted_content']
                .replace("<span style='background-color:#00FEFE'>", "<sentiment>")
                .replace("</span>", "</sentiment>"),
            'naturalLanguageExplanation': row['natural_language_explanation'],
        },
        'controlQuestion': {
            'question': row['control_question'],
            'correctAnswer': row['answer_correct'],
            'wrongAnswers': [
                row['answer_wrong_1'],
                row['answer_wrong_2'],
                row['answer_wrong_3'],
                row['answer_wrong_4'],
            ]
        }
    })

In [16]:
news_items

[{'id': 1,
  'group': 1,
  'isFalsePositive': False,
  'isTrueNegative': False,
  'isQualification': False,
  'title': '‘Miracle’ baby opens debate over possible use of centuries-old sperm',
  'subtitle': 'Technology allows sperm to be frozen longer than legal 50-year limit but poses medical and ethical questions',
  'content': 'A change of law has paved the way for more babies to be born from sperm frozen up to 50 years ago, but experts say there is no scientific reason why sperm hundreds of years old cannot be used. This week, a boy was born using sperm frozen in 1996, collected when his father was diagnosed, aged 21, with Hodgkin lymphoma, in case his treatment caused infertility. While conserving sperm for decades could theoretically enable children to be born from historical figures, this would not be possible in the UK at present, where the law prevents the creation of a sperm market and limits its use to 10 families per donor. Allan also questioned the likelihood of such a schem

In [17]:
pd.DataFrame(news_items).to_json('../src/data/news-items.json', orient='records')